In [1]:
import requests
import json
import pandas as pd

In [2]:
#Here are a few coordinates, and you have to find the fortnightly average temperature for these coordinates(longitude and latitude):
lat_long_deg_min_sec = [
('37° 52\' 5.336" S', '144° 55\' 40.452" E'),
('38° 19\' 57.125" S', '144° 54\' 17.831" E'),
('38° 51\' 34.878" S', '142° 54\' 18.281" E'), 
('37° 50\' 33.481" S', '135° 38\' 20.388" E'),
('35° 5\' 50.501" S', '128° 48\' 40.243" E'), 
('34° 20\' 7.398" S', '121° 35\' 19.757" E'),
('34° 55\' 34.99" S', '118° 45\' 4.596" E'), 
('35° 3\' 36.441" S', '115° 51\' 54.355" E'),
('30° 45\' 16.173" S', '110° 26\' 25.532" E'),
('12° 5\' 45.683" S', '106° 25\' 8.984" E'),
('6° 18\' 54.271" S', '105° 33\' 56.07" E'), 
('5° 15\' 14.261" S', '106° 13\' 37.411" E'),
('3° 42\' 37.526" S', '107° 35\' 13.608" E'), 
('2° 0\' 50.122" S', '107° 12\' 50.324" E'),
('0° 42\' 51.809" N', '105° 19\' 28.118" E'), 
('1° 18\' 56.371" N', '104° 19\' 31.582" E'),
('1° 13\' 14.328" N', '103° 52\' 6.734" E')
]

In [3]:
import re
def convert(tude):
    multiplier = 1 if tude[-1] in ['N', 'E'] else -1
    tude = re.split('° |\' |" ', tude)
    #print(tude)
    return multiplier * sum(float(x) / 60 ** n for n, x in enumerate(tude[:-1]))

In [4]:
lat_long_dec = []

for lat_long in lat_long_deg_min_sec:
    lat = convert(lat_long[0])
    long = convert(lat_long[1])
    lat_long_dec.append((lat, long))

print(lat_long_dec)

[(-37.86814888888889, 144.92790333333332), (-38.33253472222223, 144.90495305555555), (-38.85968833333334, 142.90507805555555), (-37.84263361111111, 135.63899666666666), (-35.09736138888889, 128.81117861111113), (-34.335388333333334, 121.58882138888889), (-34.92638611111111, 118.75127666666667), (-35.0601225, 115.86509861111111), (-30.7544925, 110.44042555555556), (-12.096023055555555, 106.41916222222223), (-6.315075277777778, 105.565575), (-5.2539613888888885, 106.22705861111112), (-3.710423888888889, 107.58711333333333), (-2.0139227777777777, 107.21397888888889), (0.7143913888888889, 105.32447722222221), (1.3156586111111113, 104.32543944444444), (1.2206466666666669, 103.86853722222222)]


In [5]:
CLIENT_ID1 = '1edaebb5332c44b8ac3124613212406'
CLIENT_ID2 = '50579375f5d1481e85e161222211406'

In [6]:
for lat, long in lat_long_dec:
    print(lat, long)

    # create the API request URL
    url = 'https://api.worldweatheronline.com/premium/v1/weather.ashx?key={}&q={},{}&format=json&num_of_days={}'.format(
        CLIENT_ID1,
        lat, 
        long, 
        5)
    
    response  = requests.get(url)
    forecasts  = response.json()
    
    try:
        print("Date: {}, Max Temperature: {}".format(forecasts['data']['weather'][0]['date'], forecasts['data']['weather'][0]['maxtempC']))
    except:
        print('Error: {}'.format(forecasts['data']['error'][0]))
    

-37.86814888888889 144.92790333333332
Date: 2021-07-02, Max Temperature: 13
-38.33253472222223 144.90495305555555
Date: 2021-07-02, Max Temperature: 14
-38.85968833333334 142.90507805555555
Date: 2021-07-02, Max Temperature: 11
-37.84263361111111 135.63899666666666
Date: 2021-07-02, Max Temperature: 12
-35.09736138888889 128.81117861111113
Error: {'msg': 'Unable to find any matching weather location to the query submitted!'}
-34.335388333333334 121.58882138888889
Date: 2021-07-02, Max Temperature: 13
-34.92638611111111 118.75127666666667
Date: 2021-07-02, Max Temperature: 14
-35.0601225 115.86509861111111
Date: 2021-07-02, Max Temperature: 17
-30.7544925 110.44042555555556
Error: {'msg': 'Unable to find any matching weather location to the query submitted!'}
-12.096023055555555 106.41916222222223
Date: 2021-07-02, Max Temperature: 28
-6.315075277777778 105.565575
Date: 2021-07-02, Max Temperature: 30
-5.2539613888888885 106.22705861111112
Date: 2021-07-02, Max Temperature: 30
-3.710423

In [7]:
def get_wwo_historical(lat, long, start_date, end_date):
    CLIENT_ID = '1edaebb5332c44b8ac3124613212406'
    
    # create the API request URL
    url = 'https://api.worldweatheronline.com/premium/v1/past-weather.ashx?key={}&q={},{}&format=json&date={}&enddate={}&tp=24'.format(
        CLIENT_ID,
        lat, 
        long, 
        start_date,
        end_date)
    
    response  = requests.get(url)
    forecasts  = response.json()
    
    try:
        print("Date: {}, Max Temperature: {}".format(forecasts['data']['weather'][0]['date'], forecasts['data']['weather'][0]['maxtempC']))
        return forecasts['data']['weather']
    except:
        print('Error: {}'.format(forecasts['data']['error'][0]))

In [8]:
!pip install wwo-hist

In [9]:
from wwo_hist import retrieve_hist_data

In [12]:
import os.path

if os.path.isfile('hist_weather_data.csv'):
    hist_weather_data_df = pd.read_csv('hist_weather_data.csv')
else:
    frequency = 24
    start_date = '1-JUN-2016'
    end_date = '1-JUN-2021'
    api_key = CLIENT_ID1


    hist_weather_data_df = pd.DataFrame()
    no_weather_data_list = []

    for lat, long in lat_long_dec:
        redo = True
        while redo == True:
            try:
                location_list = []
                location_list.append('{},{}'.format(lat, long))

                hist_weather_data_list = retrieve_hist_data(api_key,
                                            location_list,
                                            start_date,
                                            end_date,
                                            frequency,
                                            location_label = False,
                                            export_csv = False,
                                            store_df = True)

                # hist_weather_data_list is a list of datframes. Since we are making one call for one location, we get the first dataframe as the first element of the list
                hist_weather_data_df = hist_weather_data_df.append(hist_weather_data_list[0])
                redo = False
            except KeyError:
                no_weather_data_list.append((lat, long))
                print('Unable to find any matching weather for location {},{}'.format(lat, long))
                redo = False
            except Exception as e:
                print(e.__class__)
                if api_key == CLIENT_ID1:
                    api_key = CLIENT_ID2
                    redo = True
                else:
                    redo = False

                
    hist_weather_data_df.reset_index(inplace=True, drop=True)
    hist_weather_data_df.to_csv('hist_weather_data.csv', index=False)
        
print('\n<---- Done! ---->\n')



Retrieving weather data for -37.86814888888889,144.92790333333332


Currently retrieving data for -37.86814888888889,144.92790333333332: from 2016-06-01 to 2016-06-30
Time elapsed (hh:mm:ss.ms) 0:00:01.550258
Currently retrieving data for -37.86814888888889,144.92790333333332: from 2016-07-01 to 2016-07-31
Time elapsed (hh:mm:ss.ms) 0:00:03.139322
Currently retrieving data for -37.86814888888889,144.92790333333332: from 2016-08-01 to 2016-08-31
Time elapsed (hh:mm:ss.ms) 0:00:04.674782
Currently retrieving data for -37.86814888888889,144.92790333333332: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:00:06.124511
Currently retrieving data for -37.86814888888889,144.92790333333332: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:00:07.578635
Currently retrieving data for -37.86814888888889,144.92790333333332: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:00:09.034688
Currently retrieving data for -37.86814888888889,144.92790333333332: from 2016-12-01

Time elapsed (hh:mm:ss.ms) 0:01:26.284552
Currently retrieving data for -37.86814888888889,144.92790333333332: from 2021-04-01 to 2021-04-30
Time elapsed (hh:mm:ss.ms) 0:01:27.879529
Currently retrieving data for -37.86814888888889,144.92790333333332: from 2021-05-01 to 2021-05-31
Time elapsed (hh:mm:ss.ms) 0:01:29.400055
Currently retrieving data for -37.86814888888889,144.92790333333332: from 2021-06-01 to 2021-06-01
Time elapsed (hh:mm:ss.ms) 0:01:29.689185


Retrieving weather data for -38.33253472222223,144.90495305555555


Currently retrieving data for -38.33253472222223,144.90495305555555: from 2016-06-01 to 2016-06-30
Time elapsed (hh:mm:ss.ms) 0:00:01.386823
Currently retrieving data for -38.33253472222223,144.90495305555555: from 2016-07-01 to 2016-07-31
Time elapsed (hh:mm:ss.ms) 0:00:02.882940
Currently retrieving data for -38.33253472222223,144.90495305555555: from 2016-08-01 to 2016-08-31
Time elapsed (hh:mm:ss.ms) 0:00:04.391785
Currently retrieving data for -38.33253472

Time elapsed (hh:mm:ss.ms) 0:01:22.051652
Currently retrieving data for -38.33253472222223,144.90495305555555: from 2021-01-01 to 2021-01-31
Time elapsed (hh:mm:ss.ms) 0:01:23.580031
Currently retrieving data for -38.33253472222223,144.90495305555555: from 2021-02-01 to 2021-02-28
Time elapsed (hh:mm:ss.ms) 0:01:24.901903
Currently retrieving data for -38.33253472222223,144.90495305555555: from 2021-03-01 to 2021-03-31
Time elapsed (hh:mm:ss.ms) 0:01:26.423163
Currently retrieving data for -38.33253472222223,144.90495305555555: from 2021-04-01 to 2021-04-30
Time elapsed (hh:mm:ss.ms) 0:01:27.889432
Currently retrieving data for -38.33253472222223,144.90495305555555: from 2021-05-01 to 2021-05-31
Time elapsed (hh:mm:ss.ms) 0:01:29.473868
Currently retrieving data for -38.33253472222223,144.90495305555555: from 2021-06-01 to 2021-06-01
Time elapsed (hh:mm:ss.ms) 0:01:29.781842


Retrieving weather data for -38.85968833333334,142.90507805555555


Currently retrieving data for -38.85968833

Time elapsed (hh:mm:ss.ms) 0:01:18.378536
Currently retrieving data for -38.85968833333334,142.90507805555555: from 2020-10-01 to 2020-10-31
Time elapsed (hh:mm:ss.ms) 0:01:19.875627
Currently retrieving data for -38.85968833333334,142.90507805555555: from 2020-11-01 to 2020-11-30
Time elapsed (hh:mm:ss.ms) 0:01:21.308176
Currently retrieving data for -38.85968833333334,142.90507805555555: from 2020-12-01 to 2020-12-31
Time elapsed (hh:mm:ss.ms) 0:01:22.803680
Currently retrieving data for -38.85968833333334,142.90507805555555: from 2021-01-01 to 2021-01-31
Time elapsed (hh:mm:ss.ms) 0:01:23.854078
Currently retrieving data for -38.85968833333334,142.90507805555555: from 2021-02-01 to 2021-02-28
Time elapsed (hh:mm:ss.ms) 0:01:25.162221
Currently retrieving data for -38.85968833333334,142.90507805555555: from 2021-03-01 to 2021-03-31
Time elapsed (hh:mm:ss.ms) 0:01:26.473324
Currently retrieving data for -38.85968833333334,142.90507805555555: from 2021-04-01 to 2021-04-30
Time elapsed 

Time elapsed (hh:mm:ss.ms) 0:01:11.237502
Currently retrieving data for -37.84263361111111,135.63899666666666: from 2020-07-01 to 2020-07-31
Time elapsed (hh:mm:ss.ms) 0:01:12.672156
Currently retrieving data for -37.84263361111111,135.63899666666666: from 2020-08-01 to 2020-08-31
Time elapsed (hh:mm:ss.ms) 0:01:13.998059
Currently retrieving data for -37.84263361111111,135.63899666666666: from 2020-09-01 to 2020-09-30
Time elapsed (hh:mm:ss.ms) 0:01:15.400847
Currently retrieving data for -37.84263361111111,135.63899666666666: from 2020-10-01 to 2020-10-31
Time elapsed (hh:mm:ss.ms) 0:01:16.895315
Currently retrieving data for -37.84263361111111,135.63899666666666: from 2020-11-01 to 2020-11-30
Time elapsed (hh:mm:ss.ms) 0:01:18.336082
Currently retrieving data for -37.84263361111111,135.63899666666666: from 2020-12-01 to 2020-12-31
Time elapsed (hh:mm:ss.ms) 0:01:19.939419
Currently retrieving data for -37.84263361111111,135.63899666666666: from 2021-01-01 to 2021-01-31
Time elapsed 

...

Time elapsed (hh:mm:ss.ms) 0:00:44.699614
Currently retrieving data for -3.710423888888889,107.58711333333333: from 2019-02-01 to 2019-02-28
Time elapsed (hh:mm:ss.ms) 0:00:45.975474
Currently retrieving data for -3.710423888888889,107.58711333333333: from 2019-03-01 to 2019-03-31
Time elapsed (hh:mm:ss.ms) 0:00:47.145989
Currently retrieving data for -3.710423888888889,107.58711333333333: from 2019-04-01 to 2019-04-30
Time elapsed (hh:mm:ss.ms) 0:00:48.425543
Currently retrieving data for -3.710423888888889,107.58711333333333: from 2019-05-01 to 2019-05-31
Time elapsed (hh:mm:ss.ms) 0:00:49.797897
Currently retrieving data for -3.710423888888889,107.58711333333333: from 2019-06-01 to 2019-06-30
Time elapsed (hh:mm:ss.ms) 0:00:51.133975
Currently retrieving data for -3.710423888888889,107.58711333333333: from 2019-07-01 to 2019-07-31
Time elapsed (hh:mm:ss.ms) 0:00:52.550160
Currently retrieving data for -3.710423888888889,107.58711333333333: from 2019-08-01 to 2019-08-31
Time elapsed 

Time elapsed (hh:mm:ss.ms) 0:00:40.960053
Currently retrieving data for -2.0139227777777777,107.21397888888889: from 2018-11-01 to 2018-11-30
Time elapsed (hh:mm:ss.ms) 0:00:42.300746
Currently retrieving data for -2.0139227777777777,107.21397888888889: from 2018-12-01 to 2018-12-31
Time elapsed (hh:mm:ss.ms) 0:00:43.713802
Currently retrieving data for -2.0139227777777777,107.21397888888889: from 2019-01-01 to 2019-01-31
Time elapsed (hh:mm:ss.ms) 0:00:45.226350
Currently retrieving data for -2.0139227777777777,107.21397888888889: from 2019-02-01 to 2019-02-28
Time elapsed (hh:mm:ss.ms) 0:00:46.592266
Currently retrieving data for -2.0139227777777777,107.21397888888889: from 2019-03-01 to 2019-03-31
Time elapsed (hh:mm:ss.ms) 0:00:48.092798
Currently retrieving data for -2.0139227777777777,107.21397888888889: from 2019-04-01 to 2019-04-30
Time elapsed (hh:mm:ss.ms) 0:00:49.511024
Currently retrieving data for -2.0139227777777777,107.21397888888889: from 2019-05-01 to 2019-05-31
Time e

Time elapsed (hh:mm:ss.ms) 0:00:36.729788
Currently retrieving data for 0.7143913888888889,105.32447722222221: from 2018-08-01 to 2018-08-31
Time elapsed (hh:mm:ss.ms) 0:00:38.145362
Currently retrieving data for 0.7143913888888889,105.32447722222221: from 2018-09-01 to 2018-09-30
Time elapsed (hh:mm:ss.ms) 0:00:39.665125
Currently retrieving data for 0.7143913888888889,105.32447722222221: from 2018-10-01 to 2018-10-31
Time elapsed (hh:mm:ss.ms) 0:00:41.156960
Currently retrieving data for 0.7143913888888889,105.32447722222221: from 2018-11-01 to 2018-11-30
Time elapsed (hh:mm:ss.ms) 0:00:42.478945
Currently retrieving data for 0.7143913888888889,105.32447722222221: from 2018-12-01 to 2018-12-31
Time elapsed (hh:mm:ss.ms) 0:00:43.893602
Currently retrieving data for 0.7143913888888889,105.32447722222221: from 2019-01-01 to 2019-01-31
Time elapsed (hh:mm:ss.ms) 0:00:45.380739
Currently retrieving data for 0.7143913888888889,105.32447722222221: from 2019-02-01 to 2019-02-28
Time elapsed 

Time elapsed (hh:mm:ss.ms) 0:00:35.220539
Currently retrieving data for 1.3156586111111113,104.32543944444444: from 2018-05-01 to 2018-05-31
Time elapsed (hh:mm:ss.ms) 0:00:36.798403
Currently retrieving data for 1.3156586111111113,104.32543944444444: from 2018-06-01 to 2018-06-30
Time elapsed (hh:mm:ss.ms) 0:00:38.242546
Currently retrieving data for 1.3156586111111113,104.32543944444444: from 2018-07-01 to 2018-07-31
Time elapsed (hh:mm:ss.ms) 0:00:39.696224
Currently retrieving data for 1.3156586111111113,104.32543944444444: from 2018-08-01 to 2018-08-31
Time elapsed (hh:mm:ss.ms) 0:00:41.396242
Currently retrieving data for 1.3156586111111113,104.32543944444444: from 2018-09-01 to 2018-09-30
Time elapsed (hh:mm:ss.ms) 0:00:42.995719
Currently retrieving data for 1.3156586111111113,104.32543944444444: from 2018-10-01 to 2018-10-31
Time elapsed (hh:mm:ss.ms) 0:00:44.597436
Currently retrieving data for 1.3156586111111113,104.32543944444444: from 2018-11-01 to 2018-11-30
Time elapsed 

Time elapsed (hh:mm:ss.ms) 0:00:30.556031
Currently retrieving data for 1.2206466666666669,103.86853722222222: from 2018-02-01 to 2018-02-28
Time elapsed (hh:mm:ss.ms) 0:00:31.903080
Currently retrieving data for 1.2206466666666669,103.86853722222222: from 2018-03-01 to 2018-03-31
Time elapsed (hh:mm:ss.ms) 0:00:33.621099
Currently retrieving data for 1.2206466666666669,103.86853722222222: from 2018-04-01 to 2018-04-30
Time elapsed (hh:mm:ss.ms) 0:00:34.948040
Currently retrieving data for 1.2206466666666669,103.86853722222222: from 2018-05-01 to 2018-05-31
Time elapsed (hh:mm:ss.ms) 0:00:36.387862
Currently retrieving data for 1.2206466666666669,103.86853722222222: from 2018-06-01 to 2018-06-30
Time elapsed (hh:mm:ss.ms) 0:00:37.732814
Currently retrieving data for 1.2206466666666669,103.86853722222222: from 2018-07-01 to 2018-07-31
Time elapsed (hh:mm:ss.ms) 0:00:38.987487
Currently retrieving data for 1.2206466666666669,103.86853722222222: from 2018-08-01 to 2018-08-31
Time elapsed 

In [13]:
hist_weather_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25578 entries, 0 to 25577
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   date_time          25578 non-null  datetime64[ns]
 1   maxtempC           25578 non-null  object        
 2   mintempC           25578 non-null  object        
 3   totalSnow_cm       25578 non-null  object        
 4   sunHour            25578 non-null  object        
 5   uvIndex            25578 non-null  object        
 6   moon_illumination  25578 non-null  object        
 7   moonrise           25578 non-null  object        
 8   moonset            25578 non-null  object        
 9   sunrise            25578 non-null  object        
 10  sunset             25578 non-null  object        
 11  DewPointC          25578 non-null  object        
 12  FeelsLikeC         25578 non-null  object        
 13  HeatIndexC         25578 non-null  object        
 14  WindCh

In [14]:
hist_weather_data_df['date_time'] = pd.to_datetime(hist_weather_data_df['date_time'])
hist_weather_data_df['tempC'] = hist_weather_data_df['tempC'].astype('int')
hist_weather_data_df

,date_time,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,moon_illumination,moonrise,moonset,sunrise,...,WindGustKmph,cloudcover,humidity,precipMM,pressure,tempC,visibility,winddirDegree,windspeedKmph,location
0,2016-06-01,14,9,0.0,7.0,3,12,02:25 AM,02:30 PM,07:27 AM,...,13,64,80,0.4,1025,14,9,178,8,"-37.86814888888889,144.92790333333332"
1,2016-06-02,16,10,0.0,8.7,4,4,03:34 AM,03:10 PM,07:28 AM,...,8,6,75,0.0,1027,16,10,124,5,"-37.86814888888889,144.92790333333332"
2,2016-06-03,15,9,0.0,8.7,3,0,04:44 AM,03:53 PM,07:28 AM,...,13,33,78,1.1,1024,15,9,105,9,"-37.86814888888889,144.92790333333332"
3,2016-06-04,15,12,0.0,3.7,3,0,05:55 AM,04:41 PM,07:29 AM,...,12,90,92,12.3,1015,15,6,135,8,"-37.86814888888889,144.92790333333332"
4,2016-06-05,15,12,0.0,3.7,3,0,07:03 AM,05:33 PM,07:30 AM,...,12,89,90,0.9,1005,15,8,236,8,"-37.86814888888889,144.92790333333332"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25573,2021-05-28,31,28,0.0,9.2,6,83,08:12 PM,07:34 AM,05:56 AM,...,8,69,72,2.4,1010,31,10,163,6,"1.2206466666666669,103.86853722222222"
25574,2021-05-29,32,28,0.0,10.5,7,76,09:15 PM,08:37 AM,05:56 AM,...,12,56,69,1.7,1009,32,10,173,9,"1.2206466666666669,103.86853722222222"
25575,2021-05-30,30,28,0.0,7.8,6,69,10:14 PM,09:39 AM,05:57 AM,...,22,62,73,14.1,1009,30,9,150,14,"1.2206466666666669,103.86853722222222"
25576,2021-05-31,31,27,0.0,7.8,6,65,11:08 PM,10:36 AM,05:57 AM,...,19,69,74,9.8,1007,31,9,157,13,"1.2206466666666669,103.86853722222222"


In [15]:
grouped = hist_weather_data_df.groupby('location')
grouped

In [16]:
!pip install folium

In [17]:
!pip install vincent

In [18]:
import folium
import vincent

# To initialize vincent in the notebook
vincent.core.initialize_notebook()

In [19]:
center_lat = sum(value[0] for value in lat_long_dec) / len(lat_long_dec)
center_long = sum(value[1] for value in lat_long_dec) / len(lat_long_dec)

m = folium.Map(location=[center_lat, center_long], zoom_start=4)

for name, group in grouped:
    print(name)
    
    loc_df = group[['date_time', 'tempC']].set_index('date_time')
    
    loc_df = loc_df.resample('2W').mean()
    
    five_years_line = vincent.Line(loc_df, width=300, height=150)
    five_years_line.axis_titles(x='Date', y='tempC')
    #five_years_line.display()
    
    five_years_vega = folium.Vega(five_years_line.to_json(), width=350, height=200)
    
    lat = name.split(',')[0]
    long = name.split(',')[1]
    
    marker = folium.Marker(
            location = [lat, long],
            fill_opacity = 0.8, color=None,
            fill_color = ('#274cc9'),
            popup = folium.Popup().add_child(five_years_vega),
            tooltip = '{}, {}'.format(lat, long)
            )

    marker.add_to(m)
    
for lat, long in no_weather_data_list:
    marker = folium.Marker(
            location = [lat, long],
            fill_opacity = 0.8, color=None,
            fill_color = ('#274cc9'),
            popup = "<b>No historical weather data available.</b>",
            tooltip = '{}, {}<br>Click for more info'.format(lat, long)
            )

    marker.add_to(m)
m

-2.0139227777777777,107.21397888888889
-3.710423888888889,107.58711333333333
-34.335388333333334,121.58882138888889
-34.92638611111111,118.75127666666667
-35.0601225,115.86509861111111
-37.84263361111111,135.63899666666666
-37.86814888888889,144.92790333333332
-38.33253472222223,144.90495305555555
-38.85968833333334,142.90507805555555
-5.2539613888888885,106.22705861111112
-6.315075277777778,105.565575
0.7143913888888889,105.32447722222221
1.2206466666666669,103.86853722222222
1.3156586111111113,104.32543944444444


In [20]:
import gmaps
import gmaps.datasets
import ipywidgets as widgets

In [21]:
gmaps.__version__

'0.8.4'

In [22]:
gmaps.configure(api_key='AIXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')

In [23]:
master_df = pd.DataFrame()

for name, group in grouped:
    loc_df = group[['date_time', 'tempC']].set_index('date_time')
    
    loc_df = loc_df.resample('2W').mean()
    
    loc_df['latitude'] = name.split(',')[0]
    loc_df['latitude'] = loc_df['latitude'].astype('float')
    
    loc_df['longitude'] = name.split(',')[1]
    loc_df['longitude'] = loc_df['longitude'].astype('float')
    
    loc_df = loc_df.reset_index()
    
    #print(loc_df.head())
    
    master_df = master_df.append(loc_df)

In [24]:
master_df

,date_time,tempC,latitude,longitude
0,2016-06-05,30.600000,-2.013923,107.213979
1,2016-06-19,30.000000,-2.013923,107.213979
2,2016-07-03,30.071429,-2.013923,107.213979
3,2016-07-17,30.214286,-2.013923,107.213979
4,2016-07-31,29.357143,-2.013923,107.213979
...,...,...,...,...
127,2021-04-18,30.000000,1.315659,104.325439
128,2021-05-02,30.214286,1.315659,104.325439
129,2021-05-16,30.571429,1.315659,104.325439
130,2021-05-30,30.571429,1.315659,104.325439


In [25]:
dates = pd.date_range(master_df.date_time.min(), master_df.date_time.max(), freq='2W')

options = [date for date in dates]

selection_slider = widgets.SelectionSlider(
    options=options,
    description='Date',
    orientation='horizontal',
    layout={'width': '900px'}
)

In [26]:
day_df = master_df[master_df['date_time'] == master_df['date_time'].min()]
locations = day_df[['latitude', 'longitude']]
weights = day_df['tempC']

In [28]:
fig = gmaps.figure(layout={'width': '1000px', 'height': '600px'})
heatmap = gmaps.heatmap_layer(locations, weights=weights)
fig.add_layer(heatmap)
fig

Figure(layout=FigureLayout(height='600px', width='1000px'))

In [29]:
def print_date(date):
    day_df = master_df[master_df['date_time'].between(date, date)]
    locations = day_df[['latitude', 'longitude']]
    weights = day_df['tempC']
    heatmap.locations = locations
    heatmap.weights = weights
    print(date)

widgets.interact(
    print_date,
    date=selection_slider
);

interactive(children=(SelectionSlider(description='Date', layout=Layout(width='900px'), options=(Timestamp('20…